In [31]:
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import polars as pl
import numpy as np
import chardet
import os

### 1. Leer el fichero

In [32]:
path = os.path.join("Data", "DGT")

In [43]:
input_file = os.path.join(path,'Parque_exacto','mat_2023.txt')
columns= ['EMISIONES_CO2','FECHA_MATR','FECHA_PRIM_MATR']

parque = pl.scan_csv(input_file, separator='|').select(columns)
parque = parque.with_columns(pl.col("FECHA_MATR","FECHA_PRIM_MATR").str.to_date("%d/%m/%Y", strict=False))
parque= parque.filter(pl.col("FECHA_MATR") > pl.col("FECHA_PRIM_MATR"))

output_file = os.path.join(path,'Parque_exacto','parque_exacto_filrado_reducico.csv')
park = parque.collect()
park.write_csv(output_file)

### 2. Check the date of the exact photo

In [39]:
output_file = os.path.join(path,'Parque_exacto','parque_exacto_filrado.csv')
columns= ['EMISIONES_CO2','FECHA_MATR','FECHA_PRIM_MATR']
parque = pl.scan_csv(output_file,try_parse_dates = True).select(columns)

In [40]:
foto_exacta = parque.collect()

In [41]:
fecha_foto = foto_exacta.select(pl.max("FECHA_MATR")).item()
print(f'The newest registered car is from teh day {fecha_foto}')

The newest registered car is from teh day 2023-12-29


### 3. Read teh registrations and deregisters

tanks to @ghuls on https://github.com/pola-rs/polars/issues/3151

In [ ]:
def tramit_file_reader(file_name, cols_to_keep = ['EMISIONES_CO2','FEC_MATRICULA','CLAVE_TRAMITE','FEC_TRAMITE']):
    
    col_widths = [8,1,8,30,22,1,21,2,1,5,6,6,6,3,2,2,2,2,24,2,2,1,8,5,8,1,1,9,3,5,30,7,3,5,1,1,1,1,1,1,11,25,25,35,70,6,6,4,4,3,8,4,4,4,6,30,50,35,25,35,4,4,4,1,25,1,4,25,8]
    col_names = ["FEC_MATRICULA","COD_CLASE_MAT","FEC_TRAMITACION","MARCA_ITV","MODELO_ITV","COD_PROCEDENCIA_ITV","BASTIDOR_ITV","COD_TIPO",
    "COD_PROPULSION_ITV","CILINDRADA_ITV","POTENCIA_ITV","TARA","PESO_MAX","NUM_PLAZAS","IND_PRECINTO","IND_EMBARGO","NUM_TRANSMISIONES",
    "NUM_TITULARES","LOCALIDAD_VEHICULO","COD_PROVINCIA_VEH","COD_PROVINCIA_MAT","CLAVE_TRAMITE","FEC_TRAMITE","CODIGO_POSTAL","FEC_PRIM_MATRICULACION",
    "IND_NUEVO_USADO","PERSONA_FISICA_JURIDICA","CODIGO_ITV","SERVICIO","COD_MUNICIPIO_INE_VEH","MUNICIPIO","KW_ITV","NUM_PLAZAS_MAX","EMISIONES_CO2","RENTING", 
    "COD_TUTELA", "COD_POSESION","IND_BAJA_DEF","IND_BAJA_TEMP", "IND_SUSTRACCION","BAJA_TELEMATICA","TIPO_ITV","VARIANTE_ITV","VERSION_ITV",
    "FABRICANTE_ITV", "MASA_ORDEN_MARCHA_ITV","MASA_MAXIMA_TECNICA_ADMISIBLE_ITV","CATEGORÍA_HOMOLOGACION_EUROPEA_ITV","CARROCERIA","PLAZAS_PIE",
    "NIVEL_EMISIONES_EURO_ITV","CONSUMO_WH/KM_ITV","CLASIFICACIÓN_REGLAMENTO_VEHICULOS_ITV", "CATEGORÍA_VEHICULO_ELECTRICO","AUTONOMÍA_VEHÍCULO_ELÉCTRICO",
    "MARCA_VEHÍCULO_BASE","FABRICANTE_VEHÍCULO_BASE","TIPO_VEHÍCULO_BASE","VARIANTE_VEHÍCULO_BASE","VERSIÓN_VEHÍCULO_BASE","DISTANCIA_EJES_12_ITV",
    "VIA_ANTERIOR_ITV","VIA_POSTERIOR_ITV","TIPO_ALIMENTACION_ITV","CONTRASEÑA_HOMOLOGACION_ITV","ECO_INNOVACION_ITV","REDUCCION_ECO_ITV","CODIGO_ECO_ITV","FEC_PROCESO"
    ]
    cols_to_keep = ['EMISIONES_CO2','FEC_MATRICULA','CLAVE_TRAMITE','FEC_TRAMITE']
    slice_tuples = []
    offset = 0

    for i, separation in enumerate(col_widths):
        start = offset
        end = separation
        slice_tuples.append((start, end,col_names[i]))
        offset += separation

    df = pl.read_csv(
        file_name,
        has_header=False,
        encoding='ISO-8859-1',
        truncate_ragged_lines=True,
        new_columns=["full_str"],
    )

    exprs = [
        pl.col("full_str")
        .str.slice(start, end)
        .str.strip_chars()
        .alias(col_name)
        for start, end, col_name in slice_tuples
    ]

    df = df.with_columns(exprs).select(cols_to_keep)
    df = df.with_columns(pl.col("FEC_MATRICULA","FEC_TRAMITE").str.to_date("%d%m%Y", strict=False))
    df = df.with_columns(pl.col("EMISIONES_CO2","CLAVE_TRAMITE").str.to_integer(base = 10, strict = False))
    df = df.rename({"CLAVE_TRAMITE": "apple"})
    return df

In [11]:
bajas = os.path.join(path,"bajas","export_mensual_bajas_202501.txt")
coso = tramit_file_reader(bajas)

In [12]:
coso
#Pasar fec tramite a fec_baja y borrar la columna de clave tramite
#Concatenar todos los dataframes de la carpeta en uno y escribirlo


EMISIONES_CO2,FEC_MATRICULA,CLAVE_TRAMITE,FEC_TRAMITE
i64,date,i64,date
53,2021-09-09,6,2025-01-01
null,2016-10-04,6,2025-01-01
null,2000-01-18,6,2025-01-01
null,1988-04-27,6,2025-01-01
null,2008-02-13,6,2025-01-01
…,…,…,…
null,2010-05-06,6,2025-01-31
null,2004-07-07,3,2025-01-31
null,2008-09-12,3,2025-01-31
